In [24]:
import numpy as np
import pandas as pd
import os
import glob
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font

#from jupyter_server.utils import path2url


In [26]:
#Función para  leer e integrar archivos de todos los gestores de un tipo especifico.

def appendfiles(filenamepattern):
    #Directorio de archivos a leer
    directory = r'C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson'
    searchpattern = os.path.join(directory, filenamepattern + "_*.txt")

    filestoread = glob.glob(searchpattern)
    print(searchpattern)
    print(filestoread)

#    output_path = os.path.join(directory, "Integrated_EUtranCellFDD_files.txt")
    outputfile_name = "Integrated_" + filenamepattern + "_files.txt"
    output_path = os.path.join(directory, outputfile_name)
    print(outputfile_name, output_path)

    #Archivos de salida
    with open(output_path, "w", encoding="utf-8") as outputfile:
        for name in filestoread:
            with open(name, "r", encoding="utf-8") as f:
                outputfile.write(f.read())
                print("Agregando archivo ", name)

    print(outputfile_name)
    return(outputfile_name)


In [27]:
# Funcion para limpiar texto no deseado

def cleanfile(filename):
    #Directorio de archivo a leer
    directory = r'C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson'

    inputfile = os.path.join(directory, filename)

    cleanfile_path = os.path.join(directory, "Clean_" + filename)

    ignorelines = ["SubNetwork,", "instance(s)", "NodeId"]
 #              "NodeId	ENodeBFunctionId	EUtranCellFDDId	activePlmnList	additionalPlmnList	administrativeState	cellBarred	cellId	cellSubscriptionCapacity	channelSelectionSetSize	dlChannelBandwidth	earfcndl	earfcnul	freqBand	noOfPucchCqiUsers	noOfPucchSrUsers	operationalState	physicalLayerCellIdGroup	physicalLayerSubCellId	sectorCarrierRef	tac	timeOfLastModification	ulChannelBandwidth"]

    with open(inputfile, 'r', encoding="utf-8") as f_in:
        lines = f_in.readlines()

    lines_result = []
    for line in lines:
        if any(p in line for p in ignorelines):
            continue

        lines_result.append(line)

    with open(cleanfile_path, 'w', encoding="utf-8") as f_out:
        f_out.writelines(lines_result)

    print(f"Líneas no deseadas eliminadas. El resultado está en '{cleanfile_path}'.")


Buscando: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\EUtranCellFDD_*.txt
Archivos encontrados: ['C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\EUtranCellFDD_14.txt', 'C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\EUtranCellFDD_9.txt']
Agregado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\EUtranCellFDD_14.txt
Agregado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\EUtranCellFDD_9.txt


C:\Users\SCaracoza\AppData\Local\Temp\ipykernel_15460\2375468858.py:40: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(tsv_path, delimiter="\t", header=None)


Buscando: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\ENodeBFunction_*.txt
Archivos encontrados: ['C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\ENodeBFunction_14.txt', 'C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\ENodeBFunction_9.txt']
Agregado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\ENodeBFunction_14.txt
Agregado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\ENodeBFunction_9.txt
Archivo raw guardado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\Datos_Modified_raw.xlsx


In [29]:
# Guardar acrhivo en excel
def convert_to_excel(cleanfile):

    directory = r'C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson'
    cleanfile_path = os.path.join(directory, cleanfile)

    try:
        df = pd.read_csv(cleanfile_path, delimiter='\t', header=None)
        print("Archivo de texto leído exitosamente.")
        #Definir el nombre del archivo de salida Excel
        archivo_excel = r"C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\Converted_" + cleanfile + ".xlsx"

        #Escribir el DataFrame en el archivo Excel
        df.to_excel(archivo_excel, index=False, header=None)
        print(f"Datos convertidos y guardados en '{archivo_excel}'")

    except FileNotFoundError:
        print(f"Error: El archivo '{cleanfile}' no fue encontrado.")
    except Exception as e:
        print(f"Ocurrió un error: {e}")

✅ Archivo final listo: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\Datos_Modified.xlsx


In [31]:
# Main

appendfiles('EUtranCellFDD')
cleanfile("Integrated_EUtranCellFDD_files.txt")
convert_to_excel("Clean_Integrated_EUtranCellFDD_files.txt")


Buscando: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\EUtranCellFDD_*.txt
Archivos encontrados: ['C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\EUtranCellFDD_14.txt', 'C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\EUtranCellFDD_9.txt']
Agregado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\EUtranCellFDD_14.txt
Agregado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\EUtranCellFDD_9.txt


C:\Users\SCaracoza\AppData\Local\Temp\ipykernel_15460\2375468858.py:40: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(tsv_path, delimiter="\t", header=None)


'C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\Converted_Clean_Integrated_EUtranCellFDD_files.txt.xlsx'

In [32]:
appendfiles('ENodeBFunction')
cleanfile("Integrated_ENodeBFunction_files.txt")
convert_to_excel("Clean_Integrated_ENodeBFunction_files.txt")


Buscando: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\ENodeBFunction_*.txt
Archivos encontrados: ['C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\ENodeBFunction_14.txt', 'C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\ENodeBFunction_9.txt']
Agregado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\ENodeBFunction_14.txt
Agregado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\ENodeBFunction_9.txt


'C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\Converted_Clean_Integrated_ENodeBFunction_files.txt.xlsx'

In [33]:
appendfiles('nodeid')
cleanfile("Integrated_nodeid_files.txt")
convert_to_excel("Clean_Integrated_nodeid_files.txt")

Buscando: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\nodeid_*.txt
Archivos encontrados: ['C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\nodeid_14.txt', 'C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\nodeid_9.txt']
Agregado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\nodeid_14.txt
Agregado: C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\nodeid_9.txt


'C:\\Users\\SCaracoza\\Documents\\AT&T\\LST Cell Ran\\Ericsson\\Converted_Clean_Integrated_nodeid_files.txt.xlsx'

In [34]:
# Empezar procesamiento del archivo acomodando columnas

# Abrir archivo
#wb = load_workbook(r'C:\Users\EAlor\Documents\AT&T\LST Cell Ran\Ericsson\Converted_with_header.xlsx')
wb = load_workbook(r'C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\Converted_Clean_Integrated_EUtranCellFDD_files.txt.xlsx')
ws = wb.active

# Detectar hasta dónde llegan los datos
ultima_fila = ws.max_row

# Definir el rango de la columna B con datos
rango = f"B1:B{ultima_fila}"

# Mover la columna afuera del rango de datos principal
ws.move_range(rango, rows=0, cols=30)

# Definir nuevo rango C a W
rango = f"C1:AF{ultima_fila}"
# Mover el resto de columnas 1 posicion a la izquierda (de C a B)
ws.move_range(rango, rows=0, cols=-1)

# Guardar
wb.save(r'C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\Modified_workfile.xlsx')

In [38]:
# Insertar header

wb = load_workbook(r'C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\Modified_workfile.xlsx')
ws = wb.active

# Definir encabezados
headers = ["eNodeB Name", "CellName", "activePlmnList_mcc", "additionalPlmnList_mcc", "administrativeState", "cellBarred", "cellId", "cellSubscriptionCapacity", "channelSelectionSetSize", "dlChannelBandwidth", "earfcndl", "earfcnul", "freqBand", "noOfPucchCqiUsers", "noOfPucchSrUsers", "operationalState", "physicalLayerCellIdGroup", "physicalLayerSubCellId", "sectorCarrierRef", "tac", "timeOfLastModification", "ulChannelBandwidth", "eNBId", "eNodeB Name Unique", "LAT", "LON", "PCI", "AT&T_Site_Name", "MOCN Activo por Celda", "Al menos una celda de MOCN encendida", "MME TEF"]

# Insertar fila en la parte superior
ws.insert_rows(1)

# Escribir encabezados con orientación vertical
for col, header in enumerate(headers, start=1):
    cell = ws.cell(row=1, column=col, value=header)
    cell.font = Font(name="Aptos Narrow", size=11)
    cell.alignment = Alignment(textRotation=90, horizontal="center", vertical="bottom")

# Guardar
wb.save(r'C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\Modified_with_header.xlsx')

In [39]:
# Obtener eNBID (Columna W)
# Leer ambos archivos
# Aqui descompone el formato del header y pone la columna eNBId afuera de la tabla
df_base = pd.read_excel(r'C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\Modified_with_header.xlsx')
df_nodeb = pd.read_excel(r"C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\Converted_Clean_Integrated_ENodeBFunction_files.txt.xlsx", header=None)

df_nodeb.columns = ["NodeId", "ENodeBFunctionId", "eNBIdnew"]
# Hacer el merge (JOIN) por columna A
#df_merge = df_base.merge(
df_temp = df_base.merge(
    df_nodeb[["NodeId", "eNBIdnew"]], # solo tomamos columnas A y C de NodeB
    left_on="eNodeB Name",
    right_on="NodeId",
    how="left"
)

df_base["eNBId"] = df_temp["eNBIdnew"]
#df_merge.drop(columns=["NodeId"], inplace=True)
# Guardar resultado
df_base.to_excel(
    r"C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Ericsson\Datos_Modified.xlsx",
    index=False
)